In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import os
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
def prediction(model, image):
  #preprocess image
    
    img = cv2.imread(image) #cvt to greyscale by 0....Ada path
    img = cv2.resize(img, (256,256))
    img = img.copy()/255
  
    predictions = (model.predict(img) > 0.5).astype("int32")
    predictions = predictions.reshape(1,-1)[0]
    return predictions

def prediction_batch(model, foldername):

  #data_size = len([entry for entry in os.listdir(foldername) if os.path.isfile(os.path.join(foldername, entry))]) + len([entry for entry in os.listdir(covid_path) if os.path.isfile(os.path.join(covid_path, entry))])

  X = np.zeros((600,256,256,3))

 # normal_path = r'/content/drive/MyDrive/Covid19/Covid Dataset/Normal_Large'
  #for i in range(len([entry for entry in os.listdir(normal_path) if os.path.isfile(os.path.join(normal_path, entry))])):
  for image in os.listdir(foldername):
    image_path = os.path.join(foldername, image)
    img = cv2.imread(image_path) #cvt to greyscale by 0....Ada path
    img = cv2.resize(img, (256,256))
    X[i] = img.copy()/255
  #preprocess image
  
  predictions = (model.predict(X) > 0.5).astype("int32")
  predictions = predictions.reshape(1,-1)[0]
  return predictions

def evaluation ( labels, predictions):
    # calculate roc curve
  fpr, tpr, thresholds = roc_curve(labels, predictions)
  auc = roc_auc_score(y, predictions)
  print('AUC: %.3f' % auc)



In [3]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Covid19/covidModel.h5')

# Check its architecture
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

# Not needed to run

In [ ]:
def predict():
  data_size = len([entry for entry in os.listdir(normal_path) if os.path.isfile(os.path.join(normal_path, entry))]) + len([entry for entry in os.listdir(covid_path) if os.path.isfile(os.path.join(covid_path, entry))])

  X = np.zeros((600,256,256,3))

  normal_path = r'/content/drive/MyDrive/Covid19/Covid Dataset/Normal_Large'
  for i in range(len([entry for entry in os.listdir(normal_path) if os.path.isfile(os.path.join(normal_path, entry))])):
    img = cv2.imread(f"/content/drive/MyDrive/Covid19/Covid Dataset/Normal_Large/Normal-{i+1}.png") #cvt to greyscale by 0....Ada path
    img = cv2.resize(img, (256,256))
    X[i] = img.copy()

  covid_path = r'/content/drive/MyDrive/Covid19/Covid Dataset/Covid_Large'
  for i in range(len([entry for entry in os.listdir(covid_path) if os.path.isfile(os.path.join(covid_path, entry))])):
    img = cv2.imread(f"/content/drive/MyDrive/Covid19/Covid Dataset/Covid_Large/COVID-{i+1}.png") #cvt to greyscale by 0....Ada path
    img = cv2.resize(img, (256,256))
    X[i+100] = img.copy() # storing after 100, first 100= normal CXR

  X = X/255.0

  y = np.zeros(600)
  y[:300] = 0
  y[300:600] = 1 # 1 = +ve

  # Scoring
  predictions = (new_model.predict(X) > 0.5).astype("int32")
  predictions = predictions.reshape(1,-1)[0]
  print(classification_report(y, predictions))

  # calculate roc curve
  fpr, tpr, thresholds = roc_curve(y, predictions)
  auc = roc_auc_score(y, predictions)
  print('AUC: %.3f' % auc)


In [ ]:
predict()

19/19 [==============================] - 164s 9s/step
              precision    recall  f1-score   support

         0.0       0.82      0.54      0.65       300
         1.0       0.66      0.88      0.75       300

    accuracy                           0.71       600
   macro avg       0.74      0.71      0.70       600
weighted avg       0.74      0.71      0.70       600

AUC: 0.712


In [ ]:
y = np.zeros(600)
y[:300] = 0
y[300:600] = 1 # 1 = +ve

In [ ]:
X = np.zeros((600,256,256,3))

#normal first

for i in range(300):
  img = cv2.imread(f"/content/drive/MyDrive/Covid19/Covid Dataset/Normal_Large/Normal-{i+1}.png") #cvt to greyscale by 0....Ada path
  img = cv2.resize(img, (256,256))
  X[i] = img.copy()

for i in range(300):
  img = cv2.imread(f"/content/drive/MyDrive/Covid19/Covid Dataset/Covid_Large/COVID-{i+1}.png") #cvt to greyscale by 0....Ada path
  img = cv2.resize(img, (256,256))
  X[i+100] = img.copy() # storing after 100, first 100= normal CXR

X = X/255.0

In [ ]:
# Scoring
predictions = (new_model.predict(X) > 0.5).astype("int32")
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y, predictions))

# calculate roc curve
fpr, tpr, thresholds = roc_curve(y, predictions)
auc = roc_auc_score(y, predictions)
print('AUC: %.3f' % auc)

19/19 [==============================] - 150s 8s/step
              precision    recall  f1-score   support

         0.0       0.82      0.54      0.65       300
         1.0       0.66      0.88      0.75       300

    accuracy                           0.71       600
   macro avg       0.74      0.71      0.70       600
weighted avg       0.74      0.71      0.70       600

AUC: 0.712


In [ ]:
pred = new_model.predict(X)

19/19 [==============================] - 149s 8s/step


In [ ]:
normal_path = r'/content/drive/MyDrive/Covid19/Covid Dataset/Normal_Large'
print(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))

300
